Table as_ 22_q: Arrivals, and requests for transfer into the UK under the Dublin Regulation, by article and country of transfer

In [52]:
def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')

if is_interactive():
    %run lib/scrape_govuk.ipynb
    metadata = scrape('https://www.gov.uk/government/statistics/immigration-statistics-october-to-december-2017-data-tables')
    import pyexcel
    from io import BytesIO

    ods_files = [f for f in metadata['files']
                 if f['type'] == 'ODS' and
                 f['title'] == 'Asylum data tables immigration statistics October to December 2017 volume 5']
    assert len(ods_files) == 1, 'Should be exactly one matching ODS file'

    ods_url = ods_files[0]['url']
    ods_title = ods_files[0]['title']

    ods_file = BytesIO(session.get(ods_files[0]['url']).content)

    data = pyexcel.get_sheet(file_content=ods_file, file_type='ods', sheet_name='as_22_q', library='pyexcel-ods3')

In [53]:
df = pd.DataFrame(data.get_array())
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,"Table as_ 22_q: Arrivals, and requests for tra...",,,,,,,,,,...,,,,,,,,,,
1,Back to contents,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,Quarter,Country Transferred from,Total arrivals into the UK,Arrivals into UK: article 8.1,Arrivals into UK: article 8.2,Arrivals into UK: article 9,Arrivals into UK: article 10,Arrivals into UK: article 11,Arrivals into UK: article 12,Arrivals into UK: article 13,...,Requests for transfer into UK: article 10,Requests for transfer into UK: article 11,Request for transfer into UK: article 12,Requests for transfer into UK: article 13,Requests for transfer into UK: article 14,Requests for transfer into UK: article 15,Request for transfer into UK: article 16,Request for transfer into UK: article 17.2,Request for transfer into UK: article 18.1,Request for transfer into UK: article 20.5
4,2015 Q1,*Total,41,1,7,0,0,0,6,0,...,1,2,101,2,1,0,3,7,169,10
5,2015 Q1,Austria,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,8,0
6,2015 Q1,Belgium,1,0,0,0,0,0,0,0,...,0,1,2,0,0,0,0,0,4,0
7,2015 Q1,Bulgaria,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16,0
8,2015 Q1,Croatia,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2015 Q1,Cyprus,2,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
observations = df.iloc[4:, :]
observations.columns= df.iloc[3]
observations.head()

3,Quarter,Country Transferred from,Total arrivals into the UK,Arrivals into UK: article 8.1,Arrivals into UK: article 8.2,Arrivals into UK: article 9,Arrivals into UK: article 10,Arrivals into UK: article 11,Arrivals into UK: article 12,Arrivals into UK: article 13,...,Requests for transfer into UK: article 10,Requests for transfer into UK: article 11,Request for transfer into UK: article 12,Requests for transfer into UK: article 13,Requests for transfer into UK: article 14,Requests for transfer into UK: article 15,Request for transfer into UK: article 16,Request for transfer into UK: article 17.2,Request for transfer into UK: article 18.1,Request for transfer into UK: article 20.5
4,2015 Q1,*Total,41,1,7,0,0,0,6,0,...,1,2,101,2,1,0,3,7,169,10
5,2015 Q1,Austria,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,8,0
6,2015 Q1,Belgium,1,0,0,0,0,0,0,0,...,0,1,2,0,0,0,0,0,4,0
7,2015 Q1,Bulgaria,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16,0
8,2015 Q1,Croatia,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
new_table = pd.melt(observations,
                       ['Quarter','Country Transferred from '],
                       var_name="Total arrivals into the UK",
                       value_name="Value")

In [56]:
list(observations)

['Quarter',
 'Country Transferred from ',
 'Total arrivals into the UK ',
 'Arrivals into UK: article 8.1 ',
 'Arrivals into UK: article 8.2',
 'Arrivals into UK: article 9',
 'Arrivals into UK: article 10',
 'Arrivals into UK: article 11',
 'Arrivals into UK: article 12',
 'Arrivals into UK: article 13',
 'Arrivals into UK: article 14',
 'Arrivals into UK: article 15',
 'Arrivals into UK: article 16',
 'Arrivals into UK: article 17.2',
 'Arrivals into UK: article 18.1',
 'Arrivals into UK: article 20.5',
 'Total requests for transfers into the UK',
 'Requests for transfer into UK: article 8.1',
 'Requests for transfer into UK: article 8.2',
 'Requests for transfer into UK: article 9',
 'Requests for transfer into UK: article 10',
 'Requests for transfer into UK: article 11',
 'Request for transfer into UK: article 12',
 'Requests for transfer into UK: article 13',
 'Requests for transfer into UK: article 14',
 'Requests for transfer into UK: article 15',
 'Request for transfer into UK

In [57]:
new_table.count()

Quarter                       29316
Country Transferred from      29316
Total arrivals into the UK    29316
Value                         29316
dtype: int64

In [58]:
new_table.dropna(inplace=True)

In [59]:
new_table = new_table[new_table['Value'] != '.']

In [60]:
new_table = new_table[new_table['Value'] != 0 ]

In [61]:
new_table = new_table[new_table['Value'] != 'z']

In [62]:
new_table.count()

Quarter                       20167
Country Transferred from      20167
Total arrivals into the UK    20167
Value                         20167
dtype: int64

In [63]:
new_table['Unit'] = 'People'
new_table['Measure Type'] = 'Count'

In [64]:
new_table.dtypes

Quarter                       object
Country Transferred from      object
Total arrivals into the UK    object
Value                         object
Unit                          object
Measure Type                  object
dtype: object

In [65]:
new_table.head()

,Quarter,Country Transferred from,Total arrivals into the UK,Value,Unit,Measure Type
0,2015 Q1,*Total,Total arrivals into the UK,41,People,Count
1,2015 Q1,Austria,Total arrivals into the UK,1,People,Count
2,2015 Q1,Belgium,Total arrivals into the UK,1,People,Count
5,2015 Q1,Cyprus,Total arrivals into the UK,2,People,Count
7,2015 Q1,Denmark,Total arrivals into the UK,2,People,Count


In [66]:
new_table['Value'] = pd.to_numeric(new_table['Value'], errors='coerce').fillna(0)

In [67]:
new_table['Value'] = new_table['Value'].astype(int)

In [68]:
new_table['Period'] = new_table['Quarter'].astype(str)

In [69]:
new_table.head()

,Quarter,Country Transferred from,Total arrivals into the UK,Value,Unit,Measure Type,Period
0,2015 Q1,*Total,Total arrivals into the UK,41,People,Count,2015 Q1
1,2015 Q1,Austria,Total arrivals into the UK,1,People,Count,2015 Q1
2,2015 Q1,Belgium,Total arrivals into the UK,1,People,Count,2015 Q1
5,2015 Q1,Cyprus,Total arrivals into the UK,2,People,Count,2015 Q1
7,2015 Q1,Denmark,Total arrivals into the UK,2,People,Count,2015 Q1


In [70]:
new_table['Country Transferred'] = new_table['Country Transferred from '].str.lstrip('*')

In [71]:
new_table['Migration Flow'] = new_table['Total arrivals into the UK'].str.strip()

In [72]:
new_table['Period'] = new_table['Period'].map(lambda cell: cell.replace(' ', '-'))

In [73]:
new_table = new_table[['Period','Country Transferred','Migration Flow','Measure Type','Value','Unit']]

In [74]:
new_table.head()

,Period,Country Transferred,Migration Flow,Measure Type,Value,Unit
0,2015-Q1,Total,Total arrivals into the UK,Count,41,People
1,2015-Q1,Austria,Total arrivals into the UK,Count,1,People
2,2015-Q1,Belgium,Total arrivals into the UK,Count,1,People
5,2015-Q1,Cyprus,Total arrivals into the UK,Count,2,People
7,2015-Q1,Denmark,Total arrivals into the UK,Count,2,People


In [75]:
new_table = new_table[new_table['Value'] != 0 ]

In [76]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

new_table.to_csv(destinationFolder / ('as_22_q.csv'), index = False)

In [77]:
new_table.count()

Period                 1603
Country Transferred    1603
Migration Flow         1603
Measure Type           1603
Value                  1603
Unit                   1603
dtype: int64

In [78]:
new_table.dtypes

Period                 object
Country Transferred    object
Migration Flow         object
Measure Type           object
Value                   int64
Unit                   object
dtype: object